In [1]:
    # This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import math
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * math.log10(max_pixel / math.sqrt(mse))
    return psnr

In [2]:
# import necessary librariries
"""Import necessary libararies"""
import tensorflow as tf
import tensorflow
from tensorflow.keras.layers import Layer,InputSpec
tensorflow.config.run_functions_eagerly(True)
import cv2
from matplotlib import pyplot as plt
import numpy as np
import sys
from keras.models import Model
from keras.layers import Input, Lambda
# import tensorflow as tf

from tensorflow.compat.v1.keras.layers import BatchNormalization
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import concatenate
import numpy as np
import os
from tensorflow.compat.v1.keras.optimizers import Adam
import cv2
import h5py
from keras import backend as K
from keras.layers import Add
from tensorflow.keras.layers import Concatenate
from keras.preprocessing.image import load_img, img_to_array, array_to_img

In [3]:
"""Creating functions for necessary preprocessing and post processing"""
def preprocess(img):
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img.astype('float32')
    img = img / 255
    return img


def postprocess(img):
    """Convert numpy array to image"""
    if len(img.shape) == 2:
        img = np.expand_dims(img, axis=-1)
    img = array_to_img(img)
    return img


# def crop_image(img, d=32):
#     '''Make dimensions divisible by `d`'''

#     new_size = (img.size[0] - img.size[0] % d,
#                 img.size[1] - img.size[1] % d)

#     bbox = [
#         int((img.size[0] - new_size[0]) / 2),
#         int((img.size[1] - new_size[1]) / 2),
#         int((img.size[0] + new_size[0]) / 2),
#         int((img.size[1] + new_size[1]) / 2),
#     ]

#     img_cropped = img.crop(bbox)
#     return img_cropped


def get_noisy_image(img, sigma):
    """Adds Gaussian noise to an image."""
    img_noisy = np.clip(img + np.random.normal(scale=sigma, size=img.shape), 0, 1).astype(np.float32)
    return img_noisy


def make_noise(method, channel, sizes):
    """Creating a random image of range points"""
    if method == 'random':
        shape = (1, sizes[0], sizes[1], channel)
        noise = np.random.uniform(0, 0.1, size=shape)
#     elif method == 'meshgrid':
#         X, Y = np.meshgrid(np.arange(0, sizes[1]) / float(sizes[1] - 1),
#                            np.arange(0, sizes[0]) / float(sizes[0] - 1))
#         X = np.expand_dims(X, axis=-1)
#         Y = np.expand_dims(Y, axis=-1)
#         noise = np.concatenate([X, Y], axis=-1)
#         noise = np.expand_dims(noise, axis=0)
    return noise


def add_noise(x, sigma):
    noise = np.random.normal(0, sigma, size=x.shape)
    return x + noise


In [4]:
from PIL import Image


def load_data(path):
    img = load_img(path)
    return img


def preprocess(img):
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img.astype('float32')
    img = img / 255
    return img


def postprocess(img):
    if len(img.shape) == 2:
        img = np.expand_dims(img, axis=-1)
    img = array_to_img(img)
    return img


def crop_image(img, d=32):
    '''Make dimensions divisible by `d`'''

    new_size = (img.size[0] - img.size[0] % d,
                img.size[1] - img.size[1] % d)

    bbox = [
        int((img.size[0] - new_size[0]) / 2),
        int((img.size[1] - new_size[1]) / 2),
        int((img.size[0] + new_size[0]) / 2),
        int((img.size[1] + new_size[1]) / 2),
    ]

    img_cropped = img.crop(bbox)
    return img_cropped


def get_noisy_image(img, sigma):
    """Adds Gaussian noise to an image."""
    img_noisy = np.clip(img + np.random.normal(scale=sigma, size=img.shape), 0, 1).astype(np.float32)
    return img_noisy


def make_noise(method, channel, sizes):
    if method == 'random':
        shape = (1, sizes[0], sizes[1], channel)
        noise = np.random.uniform(0, 0.1, size=shape)
    elif method == 'meshgrid':
        X, Y = np.meshgrid(np.arange(0, sizes[1]) / float(sizes[1] - 1),
                           np.arange(0, sizes[0]) / float(sizes[0] - 1))
        X = np.expand_dims(X, axis=-1)
        Y = np.expand_dims(Y, axis=-1)
        noise = np.concatenate([X, Y], axis=-1)
        noise = np.expand_dims(noise, axis=0)
    return noise


def add_noise(x, sigma):
    noise = np.random.normal(0, sigma, size=x.shape)
    return x + noise


def low_resolution(img, factor):
    lr_size = [
        img.size[0] // factor,
        img.size[1] // factor
    ]

    img = img.resize(lr_size, Image.ANTIALIAS)
    return img


In [60]:
"""Preprocess the image"""
img1 = load_img('../input/test-image/ori.jpg')
num_iter = 5000
img = preprocess(img1)
"""Adding noise to the image"""
img = get_noisy_image(img,120 /255.)
# img=load_img('../input/test-images1/test1.png')
m=postprocess(img[0])

In [61]:
"""viewing the preprocessig steps"""
w = 10
h = 10
fig = plt.figure(figsize=(10, 10))
columns = 2
rows = 1

# prep (x,y) for extra plotting
xs = np.linspace(0, 2*np.pi, 60)  # from 0 to 2pi
ys = np.abs(np.sin(xs))           # absolute of sine

# ax enables access to manipulate each of subplots
ax = []

for i in range(rows*columns):
    # create subplot and append to ax
    ax.append( fig.add_subplot(rows, columns, i+1) )
    if(i==0):
        ax[-1].set_title("original image")  # set title

        plt.imshow(img1)
    else:
        ax[-1].set_title("Noisy Image")  # set title

        plt.imshow(m)
plt.savefig("./preprocess.png")
plt.show()  # finally, render the plot

In [62]:
PSNR(img1,img*255)

In [63]:
"""inputs to be given to Model"""
b, w, h, c = img.shape
height, width =h,w

inputs = Input(shape=(height, width, 32))
size = [height, width]
size

In [64]:
"""Making a class for reflection padding"""
"""code taken from 'https://stackoverflow.com/questions/50677544/reflection-padding-conv2d'"""
class ReflectionPadding2D(Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        self.input_spec = [InputSpec(ndim=4)]
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def compute_output_shape(self, s):
        """ If you are using "channels_last" configuration"""
        return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

    def call(self, x, mask=None):
        w_pad,h_pad = self.padding
        return tf.pad(x, [[0,0], [h_pad,h_pad], [w_pad,w_pad], [0,0] ], 'REFLECT')

def skip_layer(x):
        x=Conv2D(filters=4,padding="same",kernel_size=1,strides=(1,1))(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.2)(x)
        return x



In [65]:
"""Building the encoder decoder architechture"""

"""Encoding starts(downsampling the image)"""
x=ReflectionPadding2D(padding=(1,1))(inputs)
x = Conv2D(filters=128,kernel_size=3,strides=(2, 2))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
# down to size=256
# # ----------------------------
x=ReflectionPadding2D(padding=(1,1))(x)
x = Conv2D(filters=128,kernel_size=3,strides=(1, 1))(x)
x = BatchNormalization()(x)
x1 = LeakyReLU(alpha=0.2)(x)
# -----------------------------
# -----------------------------
x=ReflectionPadding2D(padding=(1,1))(x1)
x=  Conv2D(filters=128,kernel_size=3,strides=(2, 2))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
# down to size=128
# -----------------------------
x=ReflectionPadding2D(padding=(1,1))(x)
x = Conv2D(filters=128,kernel_size=3,strides=(1, 1))(x)
x = BatchNormalization()(x)
x2 = LeakyReLU(alpha=0.2)(x)


# # -----------------------------------
# # ------------------------------------
x=ReflectionPadding2D(padding=(1,1))(x2)
x=  Conv2D(filters=128,kernel_size=3,strides=(2, 2))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
# down to size=64
# -----------------------------------
x=ReflectionPadding2D(padding=(1,1))(x)
x = Conv2D(filters=128,kernel_size=3,strides=(1, 1))(x)
x = BatchNormalization()(x)
x3 = LeakyReLU(alpha=0.2)(x)

# # ------------------------------
# # ------------------------------
x=ReflectionPadding2D(padding=(1,1))(x3)
x=  Conv2D(filters=128,kernel_size=3,strides=(2,2))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
# down to size 32
# ------------------------------
x=ReflectionPadding2D(padding=(1,1))(x)
x = Conv2D(filters=128,kernel_size=3,strides=(1, 1))(x)
x = BatchNormalization()(x)
x4 = LeakyReLU(alpha=0.2)(x) 
# # ------------------------------
# # ------------------------------
x=ReflectionPadding2D(padding=(1,1))(x4)
x=  Conv2D(filters=128,kernel_size=3,strides=(2,2))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
# down to size 16
# ------------------------------
x=ReflectionPadding2D(padding=(1,1))(x)
x = Conv2D(filters=128,kernel_size=3,strides=(1, 1))(x)
x = BatchNormalization()(x)
x5= LeakyReLU(alpha=0.2)(x)
# # -----------------------------------------------------------------------------------
# decoding starts

x=  Conv2D(filters=128,kernel_size=1,strides=(1,1))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
# ==========================================
"""skip layer added from encoder"""
z=skip_layer(x5)
x=Concatenate(axis=3)([z,x])
# ========================================================================
# layer 2----------------------------------------------------------------
x = BatchNormalization()(x)
x = UpSampling2D(size=(2, 2),interpolation="bilinear")(x)
x=  ReflectionPadding2D(padding=(1,1))(x)
x=  Conv2D(filters=128,kernel_size=3,strides=(1,1))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x=  Conv2D(filters=128,kernel_size=1,strides=(1,1))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
# x = UpSampling2D(size=(2, 2))(x)


"""skip layer added from encoder"""
z=skip_layer(x4)
x=Concatenate(axis=3)([z,x])
# =============================layer=3
# x = BatchNormalization()(x)
x = UpSampling2D(size=(2, 2),interpolation="bilinear")(x)
x=  ReflectionPadding2D(padding=(1,1))(x)
x=  Conv2D(filters=128,kernel_size=3,strides=(2,2))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x=  Conv2D(filters=128,kernel_size=1,strides=(1,1))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x = UpSampling2D(size=(2, 2),interpolation="bilinear")(x)


"""skip layer added from encoder"""
z=skip_layer(x3)
x=Concatenate(axis=3)([z,x])
# =============================layer=4
# x = BatchNormalization()(x)
x = UpSampling2D(size=(2, 2),interpolation="bilinear")(x)
x=  ReflectionPadding2D(padding=(1,1))(x)
x=  Conv2D(filters=128,kernel_size=3,strides=(2,2))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x=  Conv2D(filters=128,kernel_size=1,strides=(1,1))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x = UpSampling2D(size=(2, 2),interpolation="bilinear")(x)


"""skip layer added from encoder"""
z=skip_layer(x2)
x=Concatenate(axis=3)([z,x])
# =============================layer=5
x = UpSampling2D(size=(2, 2),interpolation="bilinear")(x)
x=  ReflectionPadding2D(padding=(1,1))(x)
x=  Conv2D(filters=128,kernel_size=3,strides=(2,2))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x=  Conv2D(filters=128,kernel_size=1,strides=(1,1))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x = UpSampling2D(size=(2, 2),interpolation="bilinear")(x)

z=skip_layer(x1)
x=Concatenate(axis=3)([z,x])
# =============================layer=5
x = UpSampling2D(size=(2, 2),interpolation="bilinear")(x)
x=  ReflectionPadding2D(padding=(1,1))(x)
x=  Conv2D(filters=128,kernel_size=3,strides=(2,2))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x=  Conv2D(filters=128,kernel_size=1,strides=(1,1))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x = UpSampling2D(size=(2, 2),interpolation="bilinear")(x)
x=  Conv2D(filters=128,kernel_size=1,strides=(1,1))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
y = Conv2D(3, 1)(x)
# y=LeakyReLU(alpha=0.2)(y)
y = tf.keras.activations.sigmoid(y)

model = Model(inputs, y)
print(model.summary())
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-4,
#     decay_steps=300,
#     decay_rate=0.9)
model.compile(loss="mse", optimizer=Adam(lr=0.01))

In [34]:
"save model architechture"
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='./model.png')

In [ ]:
"""training the model starting with a random noise"""
sigma=1/30
z = make_noise("random", 32, (w, h))
losses = []
ms=[]
psnr=[]
for i in range(100):
    loss = model.fit(add_noise(z, sigma), img)
    losses.append(loss)
    y = model.predict_on_batch(z)
    m=PSNR(img1,y*255)
    print(m)
    psnr.append(m)
    if i % 20 == 0:
        y = model.predict_on_batch(z)
        image=postprocess(y[0])
        ms.append(image)
        plt.imshow(image)
        plt.show()

In [ ]:
for i in range(1000):
    loss = model.fit(add_noise(z, sigma), img)
    losses.append(loss)
    y = model.predict_on_batch(z)
    m=PSNR(img1,y*255)
    print(m)
    psnr.append(m)
    if i % 20 == 0:
        y = model.predict_on_batch(z)
        image=postprocess(y[0])
        ms.append(image)
        plt.imshow(image)
        plt.show()

In [56]:
for i in range(1000):
    loss = model.fit(add_noise(z, sigma), img)
    losses.append(loss)
    y = model.predict_on_batch(z)
    m=PSNR(img1,y*255)
    print(m)
    psnr.append(m)
    if i % 20 == 0:
        y = model.predict_on_batch(z)
        image=postprocess(y[0])
        ms.append(image)
        plt.imshow(image)
        plt.show()

In [38]:
w=len(losses)
w

In [39]:
x=list(range(1,w))
plt.plot(x, psnr, label = "psnr")

plt.xlabel('epochs')
# naming the y axis
plt.ylabel("psnr value")
# giving a title to my graph

plt.title('PSNR')
 
# show a legend on the plot
plt.legend()
plt.savefig("./psnr.png")
 
# function to show the plot
plt.show()

In [40]:
x=list(range(1,w))
los=[losses[i].history['loss'][0] for i in range(w-1)]
plt.plot(x, los, label = "loss")

plt.xlabel('epochs')
# naming the y axis
plt.ylabel("mse loss")
# giving a title to my graph

plt.title('loss')
 
# show a legend on the plot
plt.legend()
plt.savefig("./loss.png")
 
# function to show the plot
plt.show()

In [42]:
"""viewing the results. The end result might not be the best result"""
n=len(ms)
print(n)
w = 10
h = 10
fig = plt.figure(figsize=(10, 10))
columns = 3
rows = 3

# prep (x,y) for extra plotting
xs = np.linspace(0, 2*np.pi, 60)  # from 0 to 2pi
ys = np.abs(np.sin(xs))           # absolute of sine

# ax enables access to manipulate each of subplots
ax = []

for i in range(columns*rows):
    img = ms[n-i-2]
    # create subplot and append to ax
    ax.append( fig.add_subplot(rows, columns, i+1) )
    if(i==0):
        ax[-1].set_title("noisy image")  # set title
        plt.axis("off")
        plt.imshow(img)
    else:
        num=(n-i)*50
        ax[-1].set_title("after "+str(num)+" iterations")  # set title
        plt.axis("off")
    
        plt.imshow(img)
plt.savefig("./result.png")
plt.show()  # finally, render the plot

In [55]:
"""viewing the preprocessig steps"""
w = 15
h = 15
fig = plt.figure(figsize=(15, 15))
columns = 2
rows = 1

# prep (x,y) for extra plotting
xs = np.linspace(0, 2*np.pi, 60)  # from 0 to 2pi
ys = np.abs(np.sin(xs))           # absolute of sine

# ax enables access to manipulate each of subplots
ax = []

for i in range(rows*columns):
    # create subplot and append to ax
    ax.append( fig.add_subplot(rows, columns, i+1) )
    if(i==0):
        ax[-1].set_title("denoised image")  # set title

        plt.imshow(ms[31])
    else:
        ax[-1].set_title("Noisy Image")  # set title

        plt.imshow(img)
plt.savefig("./compare")
plt.show()  # finally, render the plot